<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Clustering + Regresión lineal

Ejemplo de aumentar los datos con clustering<br>
v1.1

In [ ]:
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

In [ ]:
if os.access('FuelConsumptionCo2.csv', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/FuelConsumptionCo2.csv > FuelConsumptionCo2.csv
    else:
        !wget FuelConsumptionCo2.csv https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/FuelConsumptionCo2.csv

### `FuelConsumption.csv`:
El dataset **`FuelConsumption.csv`** contiene el consumo específico por tipo de vehículo y la emisión estimado de dioxido de carbono (Co2) de nuevos vehículos que son venidos en canada.<br> [Dataset source](http://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64)

- **MODELYEAR** --> ejemplo 2014
- **MAKE** --> ejemplo Acura
- **MODEL** --> ejemplo ILX
- **VEHICLE CLASS** --> ejemplo SUV
- **ENGINE SIZE** --> ejemplo 4.7
- **CYLINDERS** --> ejemplo 6
- **TRANSMISSION** --> ejemplo A6
- **FUEL CONSUMPTION in CITY(L/100 km)** --> ejemplo 9.9
- **FUEL CONSUMPTION in HWY (L/100 km)** --> ejemplo 8.9
- **FUEL CONSUMPTION COMB (L/100 km)** --> ejemplo 9.2
- **CO2 EMISSIONS (g/km)** --> ejemplo 182

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
df = pd.read_csv("FuelConsumptionCo2.csv")
des = df.describe()
des.loc['Nan'] = df.isna().sum()
des.loc['%Nan'] = (df.isna().mean())*100
des

In [ ]:
df.head()

## Fin de la limpieza
Se finalizó la limpieza, no hay datos mal cargados o incompletos en este dataset

In [ ]:
print('Cantidad de datos en observacion:', df.shape[0])

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
# Analizaremos como se compartan algunos atributos de entrada contra el objetivo (las emisiones, el Co2)
pp = sns.pairplot(data=df, x_vars=['CYLINDERS', 'ENGINESIZE', 'FUELCONSUMPTION_COMB'], y_vars=['CO2EMISSIONS'], diag_kind=None, height=5)
plt.show()


Analizaremos la relación entre le consumo de combustible (FUELCONSUMPTION_COMB) y la emisión de CO2

Se puede observar que está centrada a la izquierda la distribución y no hay outliers, por lo que no habrá problema. El hecho de que la distribución esté marcada a la izquierda querra decir que hay menos datos para analizar de alta cilindrada.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax = sns.scatterplot(x=df['FUELCONSUMPTION_COMB'], y=df['CO2EMISSIONS'], color='darkBlue', ax=ax)
ax.grid('dashed')
plt.show()

# Entrenar modelo (solo regresión lineal)
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

El primer paso es obtener los datos que serán la entrada del sistema (X) y los datos que serán la salida del modelo estimador (y)

In [ ]:
X = df[['FUELCONSUMPTION_COMB']].values
y = df['CO2EMISSIONS'].values

Siguiente paso es dividir el dataset en entrenamiento (train) y evaluación (test). Utilizaremos el criterio 70%30%

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### Crear un modelo de regresión lineal

In [ ]:
from sklearn import linear_model
lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)
y_hat= lr.predict(X_test)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax = sns.scatterplot(x=df['FUELCONSUMPTION_COMB'], y=df['CO2EMISSIONS'], color='darkBlue', ax=ax, label='data')
ax = sns.scatterplot(x=X_test[:, 0], y=y_hat, color='darkGreen', ax=ax, label='linear')
ax.grid('dashed')
ax.legend()
plt.show()

# Validar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline5.png" width="1000" align="middle">

In [ ]:
from sklearn.metrics import r2_score
score = r2_score(y_test, y_hat)
print(f"Coeficiente de determinación: {score:.2f}")

# Entrenar modelo (clustering + regresión)
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

Separar las rectas en 3 conjuntos (clusters) distintos utilizando algún algoritmo de segmentación

In [ ]:
from sklearn.mixture import GaussianMixture

X_clustering = df[['FUELCONSUMPTION_COMB', 'CO2EMISSIONS']].values

from sklearn.cluster import DBSCAN
gmm = GaussianMixture(n_components=3, covariance_type='full')
gmm.fit(X_clustering)
df["consumtion type"] = gmm.predict(X_clustering)

sns.scatterplot(data=df, x='FUELCONSUMPTION_COMB', y='CO2EMISSIONS', hue='consumtion type', palette='bright')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax = sns.scatterplot(x=df['FUELCONSUMPTION_COMB'], y=df['CO2EMISSIONS'], hue=df['consumtion type'], palette='bright', ax=ax)
ax.grid('dashed')
plt.show()

Realizar una regresión lineal por cada cluster definido

In [ ]:
n_clusters = df['consumtion type'].unique()
lr_models = []

for cluster in n_clusters:

    df_cluster = df[df['consumtion type'] == cluster]

    X_clust = df_cluster[['FUELCONSUMPTION_COMB']].values
    y_clust = df_cluster['CO2EMISSIONS'].values

    X_train_clust, X_test_clust, y_train_clust, y_test_clust = train_test_split(X_clust, y_clust, test_size=0.3, random_state=42)

    lr = linear_model.LinearRegression()
    lr.fit(X_train_clust, y_train_clust)
    y_hat_clust = lr.predict(X_test_clust)

    score = r2_score(y_test_clust, y_hat_clust)
    print(f"Coeficiente de determinación: {score:.2f} para el cluster", cluster)
    lr_models.append(lr)



# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

In [ ]:
fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot()
ax = sns.scatterplot(x=df['FUELCONSUMPTION_COMB'], y=df['CO2EMISSIONS'], color='darkBlue', ax=ax, label='data')

for lr_model in lr_models:
    X_sorted = np.asanyarray(sorted(X))
    y_pred = lr_model.predict(X_sorted)
    ax = sns.lineplot(x=X_sorted[:, 0], y=y_pred, ax=ax, label='linear+cluster')

ax.grid('dashed')
ax.legend()
plt.show()

# Conclusión
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline7.png" width="1000" align="middle">

Este dataset no tiene absolutamente nada de "ruido" o imperfeccciones, por lo que con solo haber realizado múltiples rectas en vez de una con la ayuda de la segmentación previa se obtuvo un resultado 100% perfecto. Con este ejemplo alumno ya está listo para poder realizar su proyecto completo de estimación de alquileres si así lo deseara